In [58]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import qutip as qt 
from bingo.optimizer.tf_adam_optimizer import AdamOptimizer
from bingo.gate_sets import SQRDispGateSet
from bingo.optimizer.GateSynthesizer import GateSynthesizer
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
N = 20
# We initialize the ECDGateSet here
gate_set_params = {
    'N_cav' : N,
    'alpha_scale' : 1.0, #maximum |beta| for random initialization  
    'N_SQR':6
}
gate_set = SQRDispGateSet(**gate_set_params)

In [60]:
#The target oscillator state.
fock = 1
psi_t = qt.basis(N,fock) #target state

In [61]:
#the optimization options
synth_params = {
    'gateset' : gate_set,
    'N_blocks':6,
    'N_multistart' : 100, #Batch size (number of circuit optimizations to run in parallel)
    'epochs' : 100, #number of epochs before termination
    'epoch_size' : 10, #number of adam steps per epoch
    'learning_rate' : 0.01, #adam learning rate
    'term_fid' : 0.995, #terminal fidelitiy
    'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
    'beta_scale' : 3.0, #maximum |beta| for random initialization
    'initial_states' : [qt.tensor(qt.basis(2,0),qt.basis(N,0))], #qubit tensor oscillator, start in |g> |0>
    'target_states' : [qt.tensor(qt.basis(2,0), psi_t)], #end in |e> |target>.
    'name' : 'Fock %d' % fock, #name for printing and saving
    'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}
gatesynth = GateSynthesizer(**synth_params)

In [62]:
import tensorflow as tf
d = gate_set.disp_op(tf.constant([1.0]))
d.shape

TensorShape([1, 40, 40])

In [63]:
#create optimization object. 
#initial params will be randomized upon creation
opt = AdamOptimizer(gatesynth)

#print optimization info. this lives in gatesynth, since we eventually want to fully abstract away the optimizer
gatesynth.print_info()

N_blocks: 6
optimization_type: state transfer
optimization_masks: {'alphas_rho': None, 'alphas_angle': None, 'phis': None, 'thetas': None}
target_unitary: None
expectation_operators: None
target_expectation_values: None
N_multistart: 100
term_fid: 0.995
dfid_stop: 1e-06
learning_rate: 0.01
epoch_size: 10
epochs: 100
name: Fock 1
filename: None
comment: 
coherent: False
timestamps: []
do_prints: True
beta_scale: 3.0
filename: Fock 1.h5

Best circuit parameters found:
alphas:    tf.Tensor(
[ 0.69911+0.35832j  0.45899-0.45742j -0.05138+0.10999j -0.2112 -0.60446j
  0.02982-0.32038j -0.28874+0.36537j], shape=(6,), dtype=complex64)
thetas:    tf.Tensor(
[[-0.57501  2.70112 -0.07561  2.04564  1.98323 -0.8194 ]
 [-1.79168 -0.78984 -1.51515  2.82474  0.87189 -1.95746]
 [ 0.34715 -0.54948 -2.9647   0.35737  1.73333  1.02782]
 [-2.58431  2.91533  1.85143  0.07756 -3.10732 -0.41288]
 [ 2.21524  2.97483  0.04597  0.08439 -1.70672 -1.24125]
 [ 2.03427  1.1978  -2.91236  0.25356  0.12525  1.0701 ]], 

In [64]:
#run optimizer.
#note the optimizer can be stopped at any time by interrupting the python consle,
#and the optimization results will still be saved and part of the opt object.
#This allows you to stop the optimization whenever you want and still use the result.
opt.optimize()

Start time: 2022-09-15 21:54:06
 Epoch: 5 / 100 Max Fid: 0.998755 Avg Fid: 0.946864 Max dFid: 0.175968 Avg dFid: 0.061223 Elapsed time: 0:00:06.721642 Expected remaining time: 0:02:07.711207 (real part)

 Optimization stopped. Term fidelity reached.

N_blocks: 6
optimization_type: state transfer
optimization_masks: {'alphas_rho': None, 'alphas_angle': None, 'phis': None, 'thetas': None}
target_unitary: None
expectation_operators: None
target_expectation_values: None
N_multistart: 100
term_fid: 0.995
dfid_stop: 1e-06
learning_rate: 0.01
epoch_size: 10
epochs: 100
name: Fock 1
filename: None
comment: 
coherent: False
timestamps: ['2022-09-15 21:54:06']
do_prints: True
beta_scale: 3.0
filename: Fock 1.h5

Best circuit parameters found:
alphas:    tf.Tensor(
[ 0.05826-0.4819j  -0.58682+0.08445j  0.18592+0.05559j -0.41406+0.15034j
 -0.00075-0.00028j  0.06081+0.29333j], shape=(6,), dtype=complex64)
thetas:    tf.Tensor(
[[-1.11739  2.40045 -2.24229 -0.52544  1.21876  1.66286]
 [-2.95772  0.0

'2022-09-15 21:54:06'

In [ ]:
#can print info, including the best circuit found.
gatesynth.print_info() 

In [ ]:
print(gatesynth.best_fidelity())

In [ ]:
#can also get the best circuit parameters directly, could save this to a .npz file.
best_circuit =  gatesynth.best_circuit()
betas = best_circuit['betas']
phis = best_circuit['phis']
thetas = best_circuit['thetas']
print(best_circuit)